# 01 - Extract data

In this notebook, you will learn how to extract the data from the videos you have recorded.

We will use a trained model to detect `car`, `truck`, `bus`, `motorcycle` and `bicycle` objects in the video frames.

## Install the required packages

In [7]:
%pip install numpy --quiet
%pip install ultralytics --quiet
%pip install supervision --quiet
%pip install ffmpeg-python --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Load the model

In [8]:
modelPath = '../model/weights/best.pt'

In [9]:
from ultralytics import YOLO

model = YOLO(modelPath)
model.model.names

{0: 'bicycle', 1: 'bus', 2: 'car', 3: 'motorcycle', 4: 'truck'}

In [10]:
model.model.names[0]

'bicycle'

## Pre-process the video

We will convert the video into one with 8 frames per second.

In [11]:
VIDEOS_DIR = '../videos' 

In [12]:
import ffmpeg
import os

GROUP_ID = 3
POINT_ID = 1
TIME = "M"  # M: mañana, T: tarde, N: noche

filename = f"G{GROUP_ID}-P{POINT_ID}-{TIME}.mp4"

inputFilename = f"{VIDEOS_DIR}/P{POINT_ID}/{filename}"
parsedFilename = f"{VIDEOS_DIR}/parsed/{filename}"
processedFilename = f"{VIDEOS_DIR}/processed/{filename}"

if os.path.exists(parsedFilename):
    os.remove(parsedFilename)

(
    ffmpeg.input(inputFilename)
    .trim(start=0, duration=10)
    .filter("fps", fps=8, round="up")
    .output(parsedFilename)
    .run()
)

(None, None)

In [13]:
import numpy as np
import supervision as sv
from collections import OrderedDict
import pandas as pd

tracker = sv.ByteTrack()

box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

bicycle_dict = OrderedDict()
bus_dict = OrderedDict()
car_dict = OrderedDict()
motorcycle_dict = OrderedDict()
truck_dict = OrderedDict()


def callback(frame: np.ndarray, frame_index: int) -> np.ndarray:
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    labels = [
        f"#{tracker_id} {model.model.names[class_id]}"
        for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)
    ]

    for xyxy, tracker_id, class_id in zip(
        detections.xyxy, detections.tracker_id, detections.class_id
    ):
        print(f"Tracker ID: {tracker_id}, Class ID: {class_id}")
        if xyxy[2] > 1750 and xyxy[3] > 600:

            def handle_dict(x):
                if len(x) == 0:
                    x[tracker_id] = frame_index
                else:
                    last_added_id, last_added_frame = x.popitem(last=True)
                    x[last_added_id] = last_added_frame
                    if frame_index - last_added_frame > 8:
                        x[tracker_id] = frame_index

            if class_id == 0:
                handle_dict(bicycle_dict)
            elif class_id == 1:
                handle_dict(bus_dict)
            elif class_id == 2:
                handle_dict(car_dict)
            elif class_id == 3:
                handle_dict(motorcycle_dict)
            elif class_id == 4:
                handle_dict(truck_dict)

    annotated_frame = box_annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels
    )

    return trace_annotator.annotate(annotated_frame, detections=detections)

In [14]:
if os.path.exists(processedFilename):
    os.remove(processedFilename)
    
sv.process_video(
   source_path=parsedFilename,
   target_path=processedFilename,
   callback=callback
)

(
    ffmpeg.input(processedFilename)
    .output(processedFilename.split('.')[0] + "_temp" + '.mp4')
    .run()
)

os.remove(processedFilename)
os.rename(processedFilename.split('.')[0] + "_temp" + '.mp4', processedFilename)

Tracker ID: 1, Class ID: 2
Tracker ID: 2, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 4, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 6, Class ID: 2
Tracker ID: 7, Class ID: 2
Tracker ID: 1, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 2, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 4, Class ID: 2
Tracker ID: 7, Class ID: 2
Tracker ID: 1, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 2, Class ID: 2
Tracker ID: 6, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 1, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 2, Class ID: 2
Tracker ID: 6, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 1, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 2, Class ID: 2
Tracker ID: 6, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 1, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 2, Class ID: 2
Tracker ID: 6, Class ID: 2
Tracker ID: 3, Class ID: 3
Tracker ID: 1, Class ID: 2
Tracker ID: 5, Class ID: 2
Tracker ID: 2, Class ID: 2
T

In [15]:
for key, value in car_dict.items():
    print(f'Bicycle {key} detected at frame {value}')

In [16]:
timelapse = 8*60
# Create bins for car_dict and bus_dict
bins = list(range(0, timelapse*7, timelapse))


lb = [f"0{int(bins[i]/timelapse)}:00 - 0{int(bins[i+1]/timelapse)}:00" for i in range(len(bins)-1)]


# Convert car_dict OrderedDict to DataFrame
car_df = pd.DataFrame(list(car_dict.items()), columns=['group', 'car count'])
car_df['bin'] = pd.cut(car_df['car count'], bins=bins, labels=lb, include_lowest=True)


# Convert bus_dict OrderedDict to DataFrame
bus_df = pd.DataFrame(list(bus_dict.items()), columns=['group', 'bus count'])
bus_df['bin'] = pd.cut(bus_df['bus count'], bins=bins, labels=lb, include_lowest=True)


# Convert bicycle_dict OrderedDict to DataFrame
bicycle_df = pd.DataFrame(list(bicycle_dict.items()), columns=['group', 'bicycle count'])
bicycle_df['bin'] = pd.cut(bicycle_df['bicycle count'], bins=bins, labels=lb, include_lowest=True)


# Convert motorcycle_dict OrderedDict to DataFrame
motorcycle_df = pd.DataFrame(list(motorcycle_dict.items()), columns=['group', 'motorcycle count'])
motorcycle_df['bin'] = pd.cut(motorcycle_df['motorcycle count'], bins=bins, labels=lb, include_lowest=True)


# Convert truck_dict OrderedDict to DataFrame
truck_df = pd.DataFrame(list(truck_dict.items()), columns=['group', 'truck count'])
truck_df['bin'] = pd.cut(truck_df['truck count'], bins=bins, labels=lb, include_lowest=True)


# Group by 'bin' for car_dict and bus_dict
car_result = car_df['bin'].value_counts().sort_index().reset_index()
car_result.columns = ['group', 'car count']


bus_result = bus_df['bin'].value_counts().sort_index().reset_index()
bus_result.columns = ['group', 'bus count']


bicycle_result = bicycle_df['bin'].value_counts().sort_index().reset_index()
bicycle_result.columns = ['group', 'bicycle count']


motorcycle_result = motorcycle_df['bin'].value_counts().sort_index().reset_index()
motorcycle_result.columns = ['group', 'motorcycle count']


truck_result = truck_df['bin'].value_counts().sort_index().reset_index()
truck_result.columns = ['group', 'truck count']




# Merge the 5 DataFrames
result = pd.merge(car_result, bus_result, on='group', how='outer')
result = pd.merge(result, bicycle_result, on='group', how='outer')
result = pd.merge(result, motorcycle_result, on='group', how='outer')
result = pd.merge(result, truck_result, on='group', how='outer')




# Convert floats to integers
result['car count'] = result['car count'].astype(int)
result['bus count'] = result['bus count'].astype(int)
result['bicycle count'] = result['bicycle count'].astype(int)
result['motorcycle count'] = result['motorcycle count'].astype(int)
result['truck count'] = result['truck count'].astype(int)


# Display the result
result

,group,car count,bus count,bicycle count,motorcycle count,truck count
0,00:00 - 01:00,0,0,0,0,0
1,01:00 - 02:00,0,0,0,0,0
2,02:00 - 03:00,0,0,0,0,0
3,03:00 - 04:00,0,0,0,0,0
4,04:00 - 05:00,0,0,0,0,0
5,05:00 - 06:00,0,0,0,0,0
